In [ ]:
from pathlib import Path
import pandas as pd
from typing import Dict, List, Optional, Tuple
import json
from animaloc.eval.metrics import PointsMetrics

# Evaluate From CSV

This notebook computes HerdNet metrics from CSV files.


In [ ]:
detections_csv = Path("detections (1).csv")  # Generated by infer.ipynb in this folder
if not detections_csv.exists():
    raise FileNotFoundError(
        f"Missing {detections_csv.resolve()}. Run `infer.ipynb` first to create the file."
    )

detections_df = pd.read_csv(detections_csv)
detections_df.head()


In [ ]:
DEFAULT_CLASSES: Dict[int, str] = {
    1: "Topi",
    2: "Buffalo",
    3: "Kob",
    4: "Warthog",
    5: "Waterbuck",
    6: "Elephant",
}

def load_class_map(path: Optional[Path] = None) -> Dict[int, str]:
    if path is None:
        return DEFAULT_CLASSES
    data = json.loads(path.read_text())
    return {int(k): str(v) for k, v in data.items()}

def extract_points(rows: pd.DataFrame) -> Tuple[List[Tuple[float, float]], List[int], List[float]]:
    coords = [(float(row["x"]), float(row["y"])) for _, row in rows.iterrows()]
    labels = [int(row["labels"]) for _, row in rows.iterrows()]
    scores = [float(row["scores"]) for _, row in rows.iterrows()] if "scores" in rows.columns else []
    return coords, labels, scores

def evaluate_metrics(
    gt_df: pd.DataFrame,
    pred_df: pd.DataFrame,
    class_map: Dict[int, str],
    radius: float,
) -> Dict[str, object]:
    num_classes = len(class_map) + 1
    metrics = PointsMetrics(radius=radius, num_classes=num_classes)

    all_images = sorted(set(gt_df["images"]) | set(pred_df["images"]))

    for image_name in all_images:
        gt_rows = gt_df[gt_df["images"] == image_name]
        det_rows = pred_df[pred_df["images"] == image_name]

        gt_coords, gt_labels, _ = extract_points(gt_rows)
        pred_coords, pred_labels, pred_scores = extract_points(det_rows)

        est_count = [pred_labels.count(cls_id) for cls_id in range(1, num_classes)]

        metrics.feed(
            gt={"loc": gt_coords, "labels": gt_labels},
            preds={"loc": pred_coords, "labels": pred_labels, "scores": pred_scores},
            est_count=est_count,
        )

    per_class_metrics = metrics.copy()
    metrics.aggregate()

    overall = {
        "precision": metrics.precision(),
        "recall": metrics.recall(),
        "f1_score": metrics.fbeta_score(),
        "mae": metrics.mae(),
        "rmse": metrics.rmse(),
        "mse": metrics.mse(),
        "accuracy": metrics.accuracy(),
    }

    per_class = {}
    for class_id, class_name in class_map.items():
        per_class[class_name] = {
            "precision": per_class_metrics.precision(class_id),
            "recall": per_class_metrics.recall(class_id),
            "f1_score": per_class_metrics.fbeta_score(class_id),
            "mae": per_class_metrics.mae(class_id),
            "rmse": per_class_metrics.rmse(class_id),
        }

    return {
        "overall": overall,
        "per_class": per_class,
        "classes": class_map,
        "radius": radius,
    }



In [ ]:
gt_csv = Path("../../../data-delplanque/test.csv")  # Update with the ground truth CSV path
radius = 5.0
output_json = Path("metrics.json")

if not gt_csv.exists():
    raise FileNotFoundError(
        f"Ground truth CSV not found at {gt_csv.resolve()}. Update the path before continuing."
    )
gt_df = pd.read_csv(gt_csv)
class_map = load_class_map()

summary = evaluate_metrics(
    gt_df=gt_df,
    pred_df=detections_df,
    class_map=class_map,
    radius=radius,
)

print("=== Metrics from CSVs ===")
print(json.dumps(summary["overall"], indent=2))
print("Per-class F1:")
for name, scores in summary["per_class"].items():
    print(
        f"  {name:10s} -> F1: {scores['f1_score']:.3f}, Recall: {scores['recall']:.3f}, Precision: {scores['precision']:.3f}"
    )

if output_json is not None:
    output_json.parent.mkdir(parents=True, exist_ok=True)
    print(f"Metrics saved to {output_json}")

print(summary)

In [ ]:
detections_csv = Path("herdnet_baseline_phase_2_detections_overlap_160.csv")  # Generated by infer.ipynb in this folder
if not detections_csv.exists():
    raise FileNotFoundError(
        f"Missing {detections_csv.resolve()}. Run `infer.ipynb` first to create the file."
    )

detections_df = pd.read_csv(detections_csv)
detections_df.head()

gt_csv = Path("../../../data-delplanque/test.csv")  # Update with the ground truth CSV path
radius = 5.0
output_json = Path("metrics.json")

if not gt_csv.exists():
    raise FileNotFoundError(
        f"Ground truth CSV not found at {gt_csv.resolve()}. Update the path before continuing."
    )
gt_df = pd.read_csv(gt_csv)
class_map = load_class_map()

summary = evaluate_metrics(
    gt_df=gt_df,
    pred_df=detections_df,
    class_map=class_map,
    radius=radius,
)

print("=== Metrics from CSVs ===")
print(json.dumps(summary["overall"], indent=2))
print("Per-class F1:")
for name, scores in summary["per_class"].items():
    print(
        f"  {name:10s} -> F1: {scores['f1_score']:.3f}, Recall: {scores['recall']:.3f}, Precision: {scores['precision']:.3f}"
    )

if output_json is not None:
    output_json.parent.mkdir(parents=True, exist_ok=True)
    print(f"Metrics saved to {output_json}")

print(summary)